# Data Fabric
## Victor Téllez García

#### Objetivo

Este notebook es un data fabric a pequeña escala que da prioridad a la ciberseguridad de los datos.

#### Funcionamiento del Sistema

Al correr la función **iniciar_sesion()** en la celda [9], se pide al usuario su nombre, si este está listado en el registro de usuarios, se ofrecerá llenar las características de los datos a los que se desea acceder. Para esto, el usuario debe saber de antemano el nombre de la base de datos y columnas a las que se quiere acceder, esto con el fin de no revelar información adicional a la que el usuario no tiene nivel de acceso. Si el usuario cuenta con el nivel de seguridad requerida para acceder a la información solicitada, se le otorgará.

Simultaneamente se está llevando **registro** de todas las acciones y la hora y día en que fueron realizadas. En caso de que un usuario extraño intente ingresar al sistema sin un nombre de usuario válido, también se tomará registro. Las posibles acciones son las siguientes:

 1) Log in.
 2) Log out.
 3) Unable to log in.
 4) Request granted.
 5) Request denied.
 6) Invalid request.
 
Cabe mencionar que en el caso de "Invalid request." no se le hace saber al usuario, pero sí se toma registro de ello.

In [1]:
import pandas as pd
import json
import os
from datetime import datetime

In [2]:
# Funcion para registrar en el log la actividad realizada
def registrar_actividad(nombre, evento, request=None):
    
    if evento == 0:
        action = "logged_no_exitoso"
    elif evento == 1:
        action = "logged_in"
    elif evento == 2:
        action = "request_exitoso"
    elif evento == 3:
        action = "request_no_exitoso"
    elif evento == 4:
        action = "request_no_valido"
    elif evento == 5:
        action = "logged_out"

    log_file = './data/log.json'
    
    try:
        with open(log_file, 'r') as file:
            log_data = json.load(file)
    except FileNotFoundError:
        log_data = {}

    if nombre not in log_data:
        log_data[nombre] = {"activities": []}
        
    now = datetime.now()
    date = now.strftime("%Y/%m/%d %H:%M:%S")

    actividad = {"action": action, "timestamp": date}
    if request is not None:
        actividad["request"] = request

    log_data[nombre]["activities"].append(actividad)

    with open(log_file, 'w') as file:
        json.dump(log_data, file, indent=4)

In [3]:
def solicitar_nombre(user_data):
    nombre = input("Por favor, ingresa tu nombre: ")
    
    if nombre not in user_data:
        print("Nombre de usuario no valido.")
        registrar_actividad(nombre, 0, request=None)
        return False  # El usuario no existe en los datos de usuarios
    registrar_actividad(nombre, 1, request=None)
    return nombre

In [4]:
# Funcion para obtener REQUEST del usuario
def solicitar_request():
    request = {}  # Diccionario para almacenar información de las bases de datos
    while True:
        nombre_base = input("¿A qué base de datos desea ingresar? (Ingrese 'salir' para detenerse): ")
        if nombre_base.lower() == 'salir':
            break

        campos = input("Escriba una lista con los campos deseados (separado por comas sin espacios): ")
        lista_campos = [campo.strip() for campo in campos.split(",")]

        # Almacenar la información en el diccionario
        request[nombre_base] = {
            "campos": lista_campos
        }

    return request

In [5]:
# Funcion para obtener PERMISO a las bases de datos
def determinar_permiso(nombre, request, user_data, db1_data, db2_data):

    user_profile = user_data[nombre]["profile"]

    for db, db_info in request.items():
        if db == 'db1' and 'campos' in db_info:
            for campo in db_info['campos']:
                if campo in db1_data and 'level' in db1_data[campo]:
                    if user_profile not in db1_data[campo]['level']:
                        registrar_actividad(nombre, 3, request)
                        return False  # El usuario no tiene permiso para acceder a db1
                else:
                    registrar_actividad(nombre, 4, request)
                    return False  # El campo especificado en la solicitud no existe en db1_data
        elif db == 'db2' and 'campos' in db_info:
            for campo in db_info['campos']:
                if campo in db2_data and 'level' in db2_data[campo]:
                    if user_profile not in db2_data[campo]['level']:
                        registrar_actividad(nombre, 3, request)
                        return False  # El usuario no tiene permiso para acceder a db2
                else:
                    registrar_actividad(nombre, 4, request)
                    return False  # El campo especificado en la solicitud no existe en db2_data
        else:
            registrar_actividad(nombre, 4, request)
            return False # El databse especificado en la solicitud no existe
    registrar_actividad(nombre, 2, request)
    return True  # El usuario tiene permiso para acceder a todas las bases de datos en la solicitud

In [6]:
# Funcion que despliega las tablas solicitadas
def leer_archivos_csv(request):

    datos = {}
    # Procesar cada base de datos en la solicitud
    for db, info in request.items():
        if 'campos' in info:
            campos = info['campos']

            # Leer el archivo CSV correspondiente
            try:
                df = pd.read_csv(f"./data/{db}.csv")
                # Seleccionar las columnas especificadas en la solicitud
                datos[db] = df[campos].to_dict(orient='records')
            except FileNotFoundError:
                datos[db] = []  # Archivo no encontrado, guardar una lista vacía

    return datos

In [7]:
# Cargar los resultados
def cargar_desde_json(archivo_json):
    try:
        with open(archivo_json, 'r') as json_file:
            data = json.load(json_file)
        return data
    except FileNotFoundError:
        return {}

In [8]:
def iniciar_sesion():
    with open("./data/users.json", "r") as f:
        user_data = json.load(f)
        
    nombre = solicitar_nombre(user_data)
    
    if nombre == False:
        return False
    # Sesion Iniciada x=0

    respuesta = "no"
    while respuesta != "si":
        request = solicitar_request()
    
        with open("./data/db1.json", "r") as f:
            db1_data = json.load(f)
        with open("./data/db2.json", "r") as f:
            db2_data = json.load(f)

        if determinar_permiso(nombre, request, user_data, db1_data, db2_data):
            print("\nEl usuario tiene permiso para acceder a las bases de datos solicitadas.\n\nPuede encontrar esta información en dataframes.json en la carpeta de results.\n")
            dataframes = leer_archivos_csv(request)
            directory = "results"
            os.makedirs(directory, exist_ok=True)

            # Guardar el diccionario de DataFrames en un archivo JSON
            json_filename = os.path.join(directory, "dataframes.json")
            with open(json_filename, 'w') as json_file:
                json.dump(dataframes, json_file, default=lambda x: x.to_dict() if isinstance(x, pd.DataFrame) else x, indent=4)
        else:
            print("El usuario no tiene permiso para acceder a las bases de datos solicitadas.")
        
        while True:
            respuesta = input("¿Desea finalizar la sesión? (si/no): ").strip().lower()
            if respuesta == "si":
                print("Sesión finalizada.")
                registrar_actividad(nombre, 5, request=None)
                break
            elif respuesta == "no":
                print("Continuando la sesión...")
                break
            else:
                print("Respuesta no válida. Por favor, responda 'si' o 'no'.")

In [9]:
iniciar_sesion()

Por favor, ingresa tu nombre: Francisco
¿A qué base de datos desea ingresar? (Ingrese 'salir' para detenerse): db1
Escriba una lista con los campos deseados (separado por comas sin espacios): id,gender,email
¿A qué base de datos desea ingresar? (Ingrese 'salir' para detenerse): salir

El usuario tiene permiso para acceder a las bases de datos solicitadas.

Puede encontrar esta información en dataframes.json en la carpeta de results.

¿Desea finalizar la sesión? (si/no): si
Sesión finalizada.


In [10]:
# Ruta al archivo JSON que deseas cargar
dataframes_cargados = cargar_desde_json("results/dataframes.json")
dataframes = {}
# Imprimir los DataFrames
for db, data_dict in dataframes_cargados.items():
    dataframes[db] = pd.DataFrame.from_dict(data_dict)

In [11]:
dataframes["db1"]

,id,gender,email
0,1,Male,gsaylor0@trellian.com
1,2,Male,battwoul1@eventbrite.com
2,3,Female,jelsy2@census.gov
3,4,Female,aearly3@yahoo.co.jp
4,5,Agender,cfouch4@ask.com
...,...,...,...
995,996,Male,sofiellyrn@hostgator.com
996,997,Female,kboldryro@nyu.edu
997,998,Polygender,btabardrp@surveymonkey.com
998,999,Male,bhowgillrq@chicagotribune.com
